In [1]:
import re
import pandas as pd


In [2]:
bank_statement_df = pd.read_excel(r"D:\NPS_phase1\CURRENT_BANK_STMT\Bank_Kamana .xlsx",skiprows=1)

In [3]:
bank_statement_df.head()

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number
0,1,9266011200011,-56333.4,DR,IMEREMIT#1212800190,NPS-IF-3133515,034461,2023-08-28T19:33:59,NaN
1,2,9266011200011,-100.0,DR,1301820000211823-10,OL1000059632,034460,2023-08-28T19:31:59,NaN
2,3,9266011200011,-418.0,DR,NEPALREMIT#Fund Tra,NPS-IF-3133430,034459,2023-08-28T19:16:38,NaN
3,4,9266011200011,100.0,CR,9819024447,"4842247,NPS","100003986403,mob",2023-08-28T19:16:22,NaN
4,5,9266011200011,-1040.0,DR,NDPC#NPS-IF-3133420,NPS-IF-3133420,034458,2023-08-28T19:12:45,NaN


In [4]:
test_date = '2023-08-28T19:33:59'
conv_date = pd.to_datetime(test_date, format="%Y-%m-%d").date()
print(conv_date)

2023-08-28


In [6]:
bank_statement_df['Date'] = pd.to_datetime(bank_statement_df['Date'],format="%Y-%m-%dT%H:%M:%S").dt.date
bank_statement_df['Date']


0      2023-08-28
1      2023-08-28
2      2023-08-28
3      2023-08-28
4      2023-08-28
          ...    
111    2023-08-28
112    2023-08-28
113    2023-08-28
114    2023-08-28
115    2023-08-28
Name: Date, Length: 116, dtype: object

In [14]:
for index, row in bank_statement_df.iterrows():
    if "10000" in (row['Desc3']):
        id = re.findall(r'10*[0-9]{7}', row['Desc3'])
        id =  id[0].replace("10000", "")
        bank_statement_df.loc[index, 'Transaction Id'] = id
    elif "10000" in (row['Desc2']):
        id = re.findall(r'10*[0-9]{7}', row['Desc2'])
        id =  id[0].replace("10000", "")
        bank_statement_df.loc[index, 'Transaction Id'] = id
    elif "NPS-IF-" in row['Desc2']:
        id = re.findall(r'[0-9]{7}', row['Desc2'])
        bank_statement_df.loc[index, 'Transaction Id'] = id[0]
    elif "FTMS-" in row['Desc2']:
        id = re.findall(r'[0-9]{6}', row['Desc2'])
        bank_statement_df.loc[index, 'Transaction Id'] = id[0]

In [15]:
bank_statement_df['Transaction Id'].count()

115

In [16]:
bank_statement_df['Transaction Id']

0      3133515
1        59632
2      3133430
3      3986403
4      3133420
        ...   
111    3128040
112    3127998
113    3127807
114    3982639
115    3127694
Name: Transaction Id, Length: 116, dtype: object

In [28]:
bank_statement_df.head()

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number,Transaction id,Transaction Id
0,1,9266011200011,-56333.4,DR,IMEREMIT#1212800190,NPS-IF-3133515,034461,2023-08-28T19:33:59,NaN,3133515,3133515
1,2,9266011200011,-100.0,DR,1301820000211823-10,OL1000059632,034460,2023-08-28T19:31:59,NaN,59632,59632
2,3,9266011200011,-418.0,DR,NEPALREMIT#Fund Tra,NPS-IF-3133430,034459,2023-08-28T19:16:38,NaN,3133430,3133430
3,4,9266011200011,100.0,CR,9819024447,"4842247,NPS","100003986403,mob",2023-08-28T19:16:22,NaN,3986403,3986403
4,5,9266011200011,-1040.0,DR,NDPC#NPS-IF-3133420,NPS-IF-3133420,034458,2023-08-28T19:12:45,NaN,3133420,3133420


In [12]:
bank_statement_df.columns

Index(['S.N', 'Ac.Number', 'Amount', 'Txn Type', 'Desc1', 'Desc2', 'Desc3',
       'Date', 'Number', 'Transaction id'],
      dtype='object')

In [8]:
def extract_transaction_id(row, col1, col2):
            for item in (row[col1], row[col2]):
                if "NPS-IF-" in item:
                    id = re.findall(r'[0-9]{7}', item)
                    return id[0]
                elif "10000" in item:
                    id = re.findall(r'10*[0-9]{7}', item)
                    return id[0].replace("10000", "")
                elif "FTMS-" in item:
                    id = re.findall(r'[0-9]{6}', item)
                    return id[0]
                elif "eod" in item:
                    return item
            return None

        

In [9]:
bank_statement_df['Transaction id'] = bank_statement_df.apply(lambda row: extract_transaction_id(row, 'Desc2', 'Desc3'), axis=1)

In [11]:
bank_statement_df['Transaction id'].count()

115